In [ ]:
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.7/187.7 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.10.0-py3-none-any.whl size=259295 sha256=68bf652bad467a76dabcfe7f4d5f719b2ffae5f8f7e987a8ce64b4b91e848d6e
  Stored in directory: /root/.cache/pip/wheels/14/9e/82/738a3a6462074384260415f8f7c3834e9309372ff627f04a2d
Successfully built neo4j


In [ ]:
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j+s://22e118bb.databases.neo4j.io"
AUTH = ("neo4j", "7QoagVlBKkihRzP2bWBuXerFYZKC5KLTXVDHcxEMCoE")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [ ]:
def save_data_to_neo4j(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    with driver.session() as session:
        for i in range(0, len(lines), 6):
            sentence = lines[i].split(': ', 1)[1].strip()
            arg1 = lines[i+1].split(': ', 1)[1].strip().split(' (DBpedia URI: ')[0]
            arg2 = lines[i+2].split(': ', 1)[1].strip().split(' (DBpedia URI: ')[0]
            rel = lines[i+3].split(': ', 1)[1].strip()

            # Cypher query to create nodes and relationships
            query = (
                "MERGE (s:Entity {uri: $arg1})"
                "MERGE (o:Entity {uri: $arg2})"
                "MERGE (s)-[:RELATION {name: $rel}]->(o)"
            )

            session.run(query, arg1=arg1, arg2=arg2, rel=rel)


In [ ]:
if __name__ == "__main__":
    file_path = "triples_file.txt"  # Replace with the path to your data file

    # Save data to Neo4j
    save_data_to_neo4j(file_path)

    # Close the driver
    driver.close()
